In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
import time
import math 

In [ ]:
from gensim.models import Word2Vec

In [ ]:
path = "/Users/nadia/Desktop/google reviews/dic"

In [ ]:
# define function to read pickle
def read_pickle(path):
    if path[-4:] != '.pkl':
        raise ValueError
    with open(path, 'rb') as fd:
        f = pickle.load(fd)
    return f

In [ ]:
# to union each review for each shop
def union_ws_result(ckip):
    doc = []
    for ws in ckip["ws"]:
        doc.append(ws)
    return doc

In [ ]:
def clear(ckip):
    ckip_ = ckip.copy()
    pos_dic = {"A", "Na", "Nb", "Nc", "Ncd", "Nv", 
               "VA", "VAC", "VB", "VC", "VCL", "VD", "VF", "VG", "VH", "VHC", 
               "VI", "VJ", "VK", "VL", "V_2"
               }

    wss = []
    poss = []
    for ws, pos in zip(ckip_["ws"], ckip_["pos"]):
        wss_ = []
        poss_ = []
        for i, j in zip(ws, pos):
            i = i.replace(" ", "")
            if (i != "") & (len(i)>1) & (j in pos_dic):
                wss_.append(i)
                poss_.append(j)
        wss.append(wss_)
        poss.append(poss_)
        
    ckip_["ws"] = wss
    ckip_["pos"] = poss
    return ckip_


In [ ]:
data = []
# read all pickle files
for filename in os.listdir(path):
    if filename.endswith(".pkl"):
        ckip = read_pickle(os.path.join(path,filename))
        ckip = clear(ckip)
        #union result to dictionary
        result = union_ws_result(ckip)
        data.append(result)

In [ ]:
all_data = []
for i in data:
    for j in i:
        all_data.append(j)

In [ ]:
# model = Word2Vec(all_data, vector_size= 300, epochs= 10, sg=1, min_count=20, hs=1, 
#                  window=2, workers=4)

In [ ]:
#model.save("word2vec_model")

In [ ]:
model = Word2Vec.load(os.path.join("../../Google_review_code/model/","word2vec_model"))

In [ ]:
print(model)

In [ ]:
def most_similar(w2v_model, words, topn):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [ ]:
model.wv.most_similar("餐點")

In [ ]:
four_aspect = most_similar(model, ["好吃", "環境", "價格", "服務"], topn=10)
four_aspect

In [ ]:
#four_aspect.to_excel("dict.xlsx", index=False)

In [ ]:
def inter_word(w2v_model, word, topn):
    first_iter = w2v_model.wv.most_similar(word, topn=topn)
    first_iter_list = []
    for i in first_iter:
        first_iter_list.append(i[0])
                
    similar_df = pd.DataFrame()
    for first_word in first_iter_list:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(first_word, topn=topn), columns=[first_word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
                print(first_word, "not found in Word2Vec model!")
    return similar_df               

In [ ]:
b = inter_word(model,"服務", topn=100)

In [ ]:
b